In [ ]:
!pip install python-dotenv

!pip install langchain_community
!pip install langchain
!pip install langchain_experimental
!pip install langchain_chroma
!pip install langchain_openai
!pip install langchain_huggingface

# !pip install polyglot
# !pip install pyicu
# polyglot download embeddings2.ar

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
import os

# langsmith로 로그 추적 가능하니 참고할 것
os.environ["LANGCHAIN_PROJECT"] = "ODQA"
os.getenv("LANGCHAIN_PROJECT")

'ODQA'

In [3]:
from langchain_community.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers import EnsembleRetriever
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
from helper import EmbeddingHelper, KiwiBM25Retriever
#Custom_JSONLoader


############################ 문서 loading(pdf 기준) ############################
###### 문서 불러옴

FILE_PATH = "/data/ephemeral/home/donghyeok/data/wikipedia_documents.json"
helper = EmbeddingHelper()  # cache embedding (한 번 embedding 한거 저장용)
loader = JSONLoader(
    file_path=FILE_PATH, 
    jq_schema=".[].text",  
    text_content=False
)
# 문서 로딩 및 확인
docs = loader.load()
helper.show_metadata(docs)

/data/ephemeral/home/minjun/.conda/envs/minjun/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_1576087/1189191765.py:8: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from helper import EmbeddingHelper, KiwiBM25Retriever


[metadata]
['source', 'seq_num']

[examples]
source  : /data/ephemeral/home/donghyeok/data/wikipedia_documents.json
seq_num : 1


In [ ]:
#확인용
# import inspect

# print(inspect.getsource(JSONLoader))

In [4]:
############################ embedding 모델 ############################
###### embedding 할 모델 선택
from transformers import AutoModel

model_name = "intfloat/multilingual-e5-large-instruct"

# emb_model = HuggingFaceEndpointEmbeddings(
#     model=model_name,
#     task="feature-extraction",
#     huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"],
# )

emb_model = AutoModel.from_pretrained(model_name)

# embedding 내역 캐싱. helper.py 참고할 것. (default로 아마 project 폴더 내 cache 폴더 만들어서 저장될 것)
embeddings = helper.cache_embedding(emb_model)

In [5]:
############################ 문서 split ############################
###### 불러온 문서 chunking (smemantic chunker는 방법 상 embedding이 필요함)

# text splitter  (recursive, semantic 참고해서 둘 중 하나 써보길 지금은 그냥 돌게 해둠)
text_splitter_rc = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
text_splitter_sm = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=80,
)

selected_splitter = text_splitter_rc

###### 둘 다써보고싶으면 확인해보라고 남겨둠. ######
# splitters = {"Recursive": text_splitter_rc, "Semantic": text_splitter_sm}
# split_documents = {}

# for n, s in splitters.items():
#     split_documents[n] = s.split_documents(docs)


# for method, docs in split_documents.items():
#     print(f"====== {method} Splitter Results =====")
#     for i, doc in enumerate(docs[:15]):
#         print(f"[Chunk {i}]", end="\n\n")
#         print(doc.page_content)
#         print("===" * 20)


In [9]:
############################ ensemble retriever 생성 ############################


# if selected_splitter == "recursive":
#     doc_splits = text_splitter_rc.split_documents(docs)
# elif selected_splitter == "semantic":
#     doc_splits = text_splitter_sm.split_documents(docs)



doc_splits = text_splitter_rc.split_documents(docs)

print("start1")
kiwi_bm25 = KiwiBM25Retriever.from_documents(doc_splits)
print("end1")


# print("start2")

# def batch_documents(docs, batch_size=5):
#     """문서를 batch_size 크기로 나눕니다."""
#     for i in range(0, len(docs), batch_size):
#         yield docs[i:i + batch_size]

# # documents를 배치로 나눠서 Chroma에 전송
# for doc_batch in batch_documents(doc_splits, batch_size=5):
#     chroma_mmr = Chroma.from_documents(
#         documents=doc_batch,
#         collection_name="rag-chroma",
#         embedding=embeddings
#     ).as_retriever(search_type="mmr")

# print("end2")









############################ 여러개 시험해보고 싶을 때 참고할 것 ###########################
# chroma_sim = Chroma.from_documents(
#     documents=doc_splits,
#     collection_name="rag-chroma",
#     embedding=embeddings,
# ).as_retriever(search_type="mmr")

# chroma_mmr = Chroma.from_documents(
#     documents=doc_splits,
#     collection_name="rag-chroma",
#     embedding=embeddings,
# ).as_retriever(search_type="mmr")

# bm25_chroma_73 = EnsembleRetriever(
#     retrievers=[kiwi_bm25, chroma_mmr],
#     weights=[0.7, 0.3],
#     search_type="mmr",
# )

# bm25_chroma_37 = EnsembleRetriever(
#     retrievers=[kiwi_bm25, chroma_mmr],
#     weights=[0.3, 0.7],
#     search_type="mmr",
# )

# bm25_chroma_55 = EnsembleRetriever(
#     retrievers=[kiwi_bm25, chroma_mmr],
#     weights=[0.5, 0.5],
#     search_type="mmr",
# )

# retrievers = {
#     "kiwi_bm25": kiwi_bm25,
#     "chroma_sim": chroma_sim,
#     "chroma_mmr": chroma_mmr,
#     "bm25_chroma_37": bm25_chroma_37,
#     "bm25_chroma_73": bm25_chroma_73,
#     "bm25_chroma_55": bm25_chroma_55,
# }

Token indices sequence length is longer than the specified maximum sequence length for this model (1335 > 512). Running this sequence through the model will result in indexing errors


start1
end1


In [10]:
retriever = EnsembleRetriever(
    retrievers=[kiwi_bm25, kiwi_bm25],
    weights=[0.5, 0.5],
    search_type="mmr",
)

In [15]:
#리트리버출력 확인
query = "처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?"
ensemble_result = retriever.invoke(query)
bm25_result = kiwi_bm25.invoke(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]")
for doc in ensemble_result:
    print(f"Content: {doc.page_content}")
    print()


print("[BM25 Retriever]")
for doc in bm25_result:
    print(f"Content: {doc.page_content}")
    print()


[Ensemble Retriever]
Content: 4대강 정비 사업
2010년 8월 17일, 당일 방송예정이던 "4대강 수심 6m ··· 누가 밀어 붙였나?"에 국토해양부 4대강살리기 추진본부는 MBC PD 수첩이 방송을 앞두고 사전배포한 보도자료가 명백한 허위사실인데도 신문과 방송, 인터넷 등을 통해 급속하게 확산되고 있어 방송금지 가처분 신청을 서울남부지방법원에 냈다고 밝혔다.  그러나 서울남부지법 민사51부(양재영 부장판사)는 국토해양부가 낸 방송금지 가처분 신청을 기각했다. 재판부는 결정문에서, "기록만으로는 방송예정인 프로그램의 내용이 명백히 진실이 아니고 방송 목적이 공공의 이익에 부합하지 않는다고 보기 어렵다. 또한 방송이 이뤄진다고 해서 신청인에게 중대하고 회복하기 어려운 손해를 입힐 우려가 있다는 점에 대한 소명이 부족하다"라고 밝혔다.  'PD수첩' 제작진에 주장에 의하면 김재철 문화방송 사장은 임원회의에서 사규위반을 이유로 처음에 방송보류를 지시했다고 한다.  그러나 이 방송은 2010년 8월 24일에 방송된다.

Content: 이철희 장영자 어음 사기 사건은 1982년 5월 4일, 사채시장의 '큰손'으로 불리던 장영자와 남편 이철희가 어음사기 혐의로 검찰에 구속되면서 일어난 대규모 어음 사기사건이다. \n\n장영자는 국회의원과 국가안전기획부 차장을 지낸 이철희를 내세워 미모와 화려한 언변으로 고위층과 긴밀한 관계를 과시한 후 자금난을 겪고 있는 기업들에 자금지원 대가로 2배에서 최고 9배짜리 어음을 받아 이를 사채시장에 유통시키고 돈을 착복했다. 어음과 담보조로 받은 견질어음을 몽땅 시중에서 할인한 후 다시 굴리는 수법으로 6400억 원의 어음을 시중에 유통시켜 1400여 억 원을 사기로 취했다. \n\n이 사건으로 인해 소위 ‘장영자 후폭풍’이라는 정계·경제계는 물론 사회 각 분야에 엄청난 파문이 몰아쳤는데, 특히 경제계에 엄청난 파문이 일었다. 장영자는 ‘경제는 유통이다’라는 유명한 말로 항변했지만 어음이 한 바퀴 돌았을 때 어음

In [16]:
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [30]:
from datasets import load_from_disk
import pandas as pd

#dataset = load_from_disk("/data/ephemeral/home/donghyeok/data/train_dataset/")
dataset = load_from_disk("/data/ephemeral/home/donghyeok/data/test_dataset/")

#print(dataset['validation'])
data_val = dataset['validation']
df_val  = pd.DataFrame(data_val )


##########validation 용도 
# df_q = df_val[['question', 'id']]
# print(df_q)
# df_a = df_val['answers']
# answer = []
# for text in df_a :
#     answer.append(text['text'])
# df_new_a = pd.DataFrame({'text': answer}) # [글자]
####################################





In [27]:
from langchain_core.prompts import load_prompt
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.example_selectors import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts.few_shot import FewShotPromptTemplate


############################ fewshot prompt 예제 ############################

examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]


example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 여기에는 선택 가능한 예시 목록이 있습니다.
    examples,
    # 여기에는 의미적 유사성을 측정하는 데 사용되는 임베딩을 생성하는 임베딩 클래스가 있습니다.
    OpenAIEmbeddings(),
    # 여기에는 임베딩을 저장하고 유사성 검색을 수행하는 데 사용되는 VectorStore 클래스가 있습니다.
    Chroma,
    # 이것은 생성할 예시의 수입니다.
    k=1,
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"

selected_examples = example_selector.select_examples({"question": question})


example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="""
    Question: {question}
    Answer: {answer}
    """,
)

prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)


############################ generator 및 chain 생성 ############################
model = ChatOpenAI(
    temperature=0.0,
    model_name="gpt-4o",
)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [28]:
question = "처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?"

# invoke를 사용해 입력을 체인에 넘김
result = chain.invoke({"question": question})

print(result)

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [ ]:
### test에 대한 답 generate 
import json
predictions = {}
for index, row in df_val.iterrows():
    result = chain.invoke({"question": row['question']})
    predictions[row['id']] = result

# 제출용 json 생성
with open('predictions.json', 'w', encoding='utf-8') as f:
    json.dump(predictions, f, ensure_ascii=False, indent=4)